# Quickstart: Compare runs, Choose a model and deploy it to a REST API

In this quickstart, you will:

- Run a hyper parameter sweep on a training script.
- Compare the results of the runs in the MLFlow UI.
- Choose the best run and register it as a model. 
- Deploy the model to a REST Api. 
- Build a container image suitable for deployment to a cloud platform. 

In [23]:
import keras
import numpy as np
import pandas as pd
import tensorflow as tf

In [24]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [25]:
## Load the dataset

data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", 
    sep=';'
    )

In [26]:
# Split the data into train,validation and test sets


train, test = train_test_split(data, test_size=0.25, random_state=42)

In [27]:
train_x = train.drop(columns=["quality"], axis=1).values
train_y = train["quality"].values.ravel()

In [28]:
# Test set

test_x = test.drop(columns=["quality"], axis=1).values
test_y = test["quality"].values.ravel()

In [29]:
# Splitting into train and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [30]:
signature = infer_signature(train_x, train_y)

In [31]:
# Build model


def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):

    # Normalize the data
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)

    # Define model architecture
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1)
        ]
    )

    # Compile the model
    model.compile(
        optimizer=keras.optimizers.SGD(learning_rate=params["lr"],momentum=params["momentum"]),
        
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()]
    )

    # Train the model with lr and momentum params which MLFLOW tracking
    with mlflow.start_run(nested=True):

        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )

        # Evaluate the model 
        eval_result = model.evaluate(valid_x, valid_y, verbose=0)

        eval_rmse = eval_result[1]

        # Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log the model
        mlflow.tensorflow.log_model(
            model,
            "model",
            signature=signature
        )

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}




In [32]:
# define an objective function
def objective(params):
    # MLFlow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y
    )
    return result
   

In [33]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}

In [37]:
mlflow.set_experiment("wine-quality")
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")


2025/05/23 13:17:40 INFO mlflow.tracking.fluent: Experiment with name 'wine-quality' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 9s 205ms/step - loss: 33.3843 - root_mean_squared_error: 5.7779
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 32.1334 - root_mean_squared_error: 5.6677 - val_loss: 24.2379 - val_root_mean_squared_error: 4.9232

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 22.9341 - root_mean_squared_error: 4.7890
29/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 21.9085 - root_mean_squared_error: 4.6791 
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20.7079 - root_mean_squared_error: 4.5463 - val_loss: 12.0771 - val_root_mean_squared_error: 3.4752

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 11.7611 - root_mean_squared_error: 3.4295
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9.8230 - root_mean_squared_error: 3.1280 - val_loss: 5.0735 - val_root_mean_squared_error: 2.2524

  0%|          | 0/4 [00:00<?, ?

2025/05/23 13:17:47 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 8s 194ms/step - loss: 34.5027 - root_mean_squared_error: 5.8739
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.5969 - root_mean_squared_error: 5.8819 - val_loss: 33.0422 - val_root_mean_squared_error: 5.7482

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 32.0707 - root_mean_squared_error: 5.6631
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 32.3248 - root_mean_squared_error: 5.6855 - val_loss: 31.0357 - val_root_mean_squared_error: 5.5710

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 31.2028 - root_mean_squared_error: 5.5859
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 30.2162 - root_mean_squared_error: 5.4969 - val_loss: 29.1487 - val_root_mean_squared_error: 5.3990

 25%|██▌       | 1/4 [00:07<00:22,  7.39s/

2025/05/23 13:17:54 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



...

Epoch 1/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 10s 233ms/step - loss: 27.2874 - root_mean_squared_error: 5.2237
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9.2414 - root_mean_squared_error: 2.9401 - val_loss: 0.7975 - val_root_mean_squared_error: 0.8931

Epoch 2/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.5915 - root_mean_squared_error: 0.7691
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6507 - root_mean_squared_error: 0.8063 - val_loss: 0.5557 - val_root_mean_squared_error: 0.7455

Epoch 3/3                                                                      

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.3064 - root_mean_squared_error: 0.5535
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5114 - root_mean_squared_error: 0.7140 - val_loss: 0.5384 - val_root_mean_squared_error: 0.7337

 75%|███████▌  | 3/4 [00:22<00:07,  7.14s/trial, 

2025/05/23 13:18:08 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.



100%|██████████| 4/4 [00:28<00:00,  7.09s/trial, best loss: 0.7337256669998169]


2025/05/23 13:18:14 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Best parameters: {'lr': np.float64(0.008157193719046114), 'momentum': np.float64(0.9233169085934034)}
Best eval rmse: 0.7337256669998169


In [40]:
# Inferencing

# Load model into memory using pyfunc
model = mlflow.pyfunc.load_model(model_uri='runs:/a798f8024a44485b9a1982081677565a/model')

# Predict directly
predictions = model.predict(test_x)

print(predictions)


39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 899us/step
[[5.9251595]
 [7.0437355]
 [6.70747  ]
 ...
 [6.4317055]
 [6.9361377]
 [6.0192413]]


In [41]:
# Register in the model registry
model_uri = 'runs:/a798f8024a44485b9a1982081677565a/model'
mlflow.register_model(model_uri, "wine-quality-model")

Successfully registered model 'wine-quality-model'.
Created version '1' of model 'wine-quality-model'.


<ModelVersion: aliases=[], creation_timestamp=1747992411778, current_stage='None', description=None, last_updated_timestamp=1747992411778, name='wine-quality-model', run_id='a798f8024a44485b9a1982081677565a', run_link=None, source='file:///Users/mohammedanas/Learning/Machine%20Learning/Machine-learning/MLFlow%20Starter/2-DL-Project/mlruns/722027470082049981/a798f8024a44485b9a1982081677565a/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>